In [1]:
# Importation des librairies
import json
from pathlib import Path
from datetime import datetime, timedelta
import requests
import re
from PIL import Image, ExifTags
Image.MAX_IMAGE_PIXELS = 933120000
from io import BytesIO
import spacy
import pandas as pd

# NASA Astronomy Picture of the Day (APOD)<br>Création du dataset

## Sommaire

* [Présentation](#présentation)
* [Adresse et clé de l'API](#adresse-et-clé-de-lapi)
* [Dossiers de destination des fichiers JSON](#dossiers-de-destination-des-fichiers-json)
* [Définition des fonctions nécessaires](#définition-des-fonctions-nécessaires)
    * [Requêtes et sauvegarde des données brutes](#requêtes-et-sauvegarde-des-données-brutes)
    * [Nettoyage du texte](#nettoyage-du-texte)
    * [Ajout des information relatives à l'image](#ajout-des-information-relative-à-limage)
    * [Ajout des information relatives au texte](#ajout-des-information-relative-au-texte)
    * [Enregistrement du *dataset* au format CSV](#enregistrement-du-dataset-au-format-csv)
* [Exécution du code](#exécution-du-code)

## Présentation

Chaque jour depuis juin 1995, sur son site [*Astronomy Picture of the Day*](https://apod.nasa.gov/apod/), la NASA présente une image différente de notre univers accompagnée d'une brève explication rédigée par un astronome professionnel.

Les métadonnées associées à chaque image sont accessibles *via* l'API dédiée mise à diposition sur le portail [NASA Open APIs](https://api.nasa.gov/).

Ce sont ces données qui serviront de base à ce projet. Elles seront ensuite nettoyées et enrichies de nouvelles *variables* pour créer le *dataset* final.

>Ce *notebook* contient le code détaillé de l'ensemble de ces étapes.

## Adresse et clé de l'API

On commence par initialiser l'adresse de l'API ainsi que la clé personnelle.

Si aucune clé n'est fournie, on utilise la clé de démo dont les limites sont :
* horaire : 30 requêtes par adresse IP par hour ;
* quotidienne : 50 requêtes par adresse IP par jour.

In [2]:
# Définit l'adresse de l'API
API_URL = 'https://api.nasa.gov/planetary/apod'

# Définit la clé de l'API contenue dans le fichier JSON externe
p = Path.cwd()
q = p / 'api_key.json'

if q.exists():
    f = open('./api_key.json')
    API_KEY = json.load(f)['api_key']
    f.close()

if not API_KEY:
    API_KEY = 'DEMO_KEY'

## Dossiers de destination des fichiers JSON

A chaque étape du *process*, des fichiers JSON sont générés pour chacune des dates de la période définie dans la requête.

>Ce choix n'est pas forcément le plus judicieux en terme d'écriture disque mais ce projet a vocation a être transposé dans **Amazon AWS** et le but est de travailler sur la gestion des ***buckets* S3** et des **Lambda**.

Ces fichiers sont enregistrés des dossiers ayant la structure suivante :<br>
.<br>
└── json/<br>
&nbsp;&nbsp;&nbsp;&nbsp;├── raw<br>
&nbsp;&nbsp;&nbsp;&nbsp;├── level-0<br>
&nbsp;&nbsp;&nbsp;&nbsp;├── level-1<br>
&nbsp;&nbsp;&nbsp;&nbsp;└── level-2

In [24]:
# Définit la structure des dossiers de destination des fichiers JSON
JSON_FOLDER = 'json'
RAW_FOLDER = 'raw'
LEVEL0_FOLDER = 'level-0'
LEVEL1_FOLDER = 'level-1'
LEVEL2_FOLDER = 'level-2'
CSV_FOLDER = 'csv'

# Crée les dossiers
folders_list = [RAW_FOLDER, LEVEL0_FOLDER, LEVEL1_FOLDER, LEVEL2_FOLDER]

for folder in folders_list:
    p = Path.cwd()
    q = p / JSON_FOLDER / folder
    if not q.exists():
        q.mkdir(parents=True, exist_ok=False)
        print(f'''Le dossier '{folder}' a été créé avec succès.''')
    else:
        print(f'''Le dossier '{folder}' existe déjà.''')

Le dossier 'raw' existe déjà.
Le dossier 'level-0' existe déjà.
Le dossier 'level-1' existe déjà.
Le dossier 'level-2' existe déjà.


## Définition des fonctions nécessaires

Cette partie contient les fonctions nécessaires à chaque étape du *process*.

### Requêtes et sauvegarde des données brutes

Pour chaque date comprise dans la période définie par `start_date` et `end_date`, une requête est envoyée à l'API et le résultat est enregistré dans un fichier JSON individuel (*niveau RAW*).

In [34]:
def get_raw_json_from_apod_api(
        start_date,
        end_date,
        api_key=API_KEY,
        api_url = API_URL
    ):

    '''
    Fonction permettant de requêter sur l'API APOD de la NASA et d'enregistrer
    le résultat de chaque requête dans dans un fichier JSON individuel.

    Paramètres :
    -
    - start_date : la date (YYYY-MM-DD) à partir de laquelle récupérer les 
    informations ;
    - end_date : la date (YYYY-MM-DD) à partir de laquelle arrêter de récupérer 
    les informations ;
    - api_key : la clé pour accéder à l'API ;
    - api_url : l'adresse de l'API.
    '''    

    # Convertit les dates au format date si elles ne sont pas vides
    if start_date and end_date:
        # Vérifie la validité du format des dates
        try:
            start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
            end_date = datetime.strptime(end_date, '%Y-%m-%d').date()
        except ValueError:
            print('Erreur : Les dates ne sont pas au format YYYY-MM-DD.')
    else:
        print("Erreur : Les dates ne doivent pas être vides.")


    # Définit le delta des périodes (attention si delta >  90 jours)
    delta = timedelta(days=90)

    # Boucle sur la période de dates
    while start_date <= end_date:
        end_of_period = min(start_date + delta, end_date)
        
        # Définit les paramètres de la requête
        params = {
            'api_key': api_key,
            'start_date': start_date.strftime('%Y-%m-%d'),
            'end_date': end_of_period.strftime('%Y-%m-%d')
        }

        # Effectue la requête
        r = requests.get(api_url, params=params)

        # Boucle sur chaque élément du résultat de la requête
        for i in range (len(r.json())):
            
            # Définit le nom du fichier de destination
            p = Path.cwd()
            q = p / JSON_FOLDER / RAW_FOLDER / (r.json()[i]['date']+'.json')
            
            # Définit le dictionnaire à écrire
            data = r.json()[i]

            # Crée le(s) fichier(s) et écrit les données de la requêtes
            with open(q, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=True, indent=4)

        start_date += delta

### Nettoyage du texte

Pour chaque fichier enregistré au *niveau RAW*, le texte des variables `copyright`, `explanation` et `title` est nettoyé. Une variable unique contenant l'url du media est créée et les variables inutiles sont supprimés. Enfin, un nouveau fichier est généré au *niveau level 0*.

In [5]:
def clean_json_file(file_to_process):
    '''
    Fonction de nettoyage des fichiers JSON issus des requêtes sur l'API APOD
    de la NASA.

    Paramètre :
    -
    - file_to_process : le fichier JSON à traiter.
    '''

    # Importe le fichier
    with open(file_to_process, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Définit les clés pour lesquelles nettoyer le texte
    keys_to_clean = ['copyright', 'explanation', 'title']

    # Boucle sur les clés à nettoyer et applique les règles définies
    for key in keys_to_clean:
        if key in data:
            # Supprime le caractère de nouvelle ligne
            data[key] = re.sub('\n', '', data[key])
            # Règle d'espace pour '.' et ','
            data[key] = re.sub(r'\s*([.,])\s*', r'\1 ', data[key])
            # Règle d'espace pour '?', '!', ':'
            data[key] = re.sub(r'\s*([;!?:])\s*', r' \1 ', data[key])
            # Règle d'espace '(', '[' et '{'
            data[key] = re.sub(r'\s*([(\[{])\s*', r' \1', data[key])
            # Règle d'espace ')', ']' et '}'
            data[key] = re.sub(r'\s*([)\]}])\s*', r'\1 ', data[key])
            # Règle d'espace '-' et '/'
            data[key] = re.sub(r'\s*([-/])\s*', r'\1', data[key])
            # Règle d'espace '--'
            data[key] = re.sub(r'--', r' -- ', data[key])
            # Supprime les espaces superflus
            data[key] = re.sub(r'\s+', ' ', data[key])
            # Supprimer 'Credit and Copyright : '
            data[key] = re.sub(r'Credit and Copyright : ', '', data[key])
            # Supprimer 'Credit : '
            data[key] = re.sub(r'Credit : ', '', data[key]) 
            # Supprimer l'espace à la fin de la chaîne de caractères
            data[key] = data[key].rstrip()

    # Supprime 'explanation' de 'copyright' (bug sur images de 1995)
    if 'copyright' in data and 'Explanation' in data['copyright']:
        data['copyright'] = data['copyright'].split(' Explanation')[0]

    # Définit les extensions de fichiers à exclure
    extensions_to_exclude = ['mov', 'mpg', 'wmv', 'avi', 'mp4', 'mkv']

    # Crée un lien unique 'media_url' dans le dictionnaire
    if data['media_type'] == 'image':
        if ('hdurl' in data
            and data['hdurl'].split('.')[-1] not in extensions_to_exclude):
            data['media_url'] = data['hdurl']
        elif data['hdurl'].split('.')[-1] in extensions_to_exclude:
            data['media_url'] = data['url']
        elif 'hdurl' not in data:
            data['media_url'] = data['url']
    elif data['media_type'] == 'video':
        data['media_url'] = data['url']

    # Définit les clés à supprimer
    keys_to_remove = ['hdurl', 'service_version', 'url']

    # Supprime les clés
    for key in keys_to_remove:
        if key in data:
            del data[key]


    # Exporte le fichier dans le dossier de destination
    p = Path.cwd()
    q = p / JSON_FOLDER / LEVEL0_FOLDER / file_to_process.name

    with open(q, 'w', encoding='utf-8') as of: 
        json.dump(data, of, indent=4)


### Ajout des information relative à l'image

Pour chaque fichier enregistré au *niveau level 0*, et avec **Request** et **Pillow**, on ouvre l'image à partir du lien contenu dans `media_url`. On extrait ensuite les informations de base de l'image puis on extrait les *tags* EXIF. Enfin, un nouveau fichier est enregistré au *niveau level 1*.

In [6]:
def add_image_features_to_json(file_to_process):
    '''
    Fonction permettant d'enrichir les données des fichiers JSON issus des
    requêtes sur l'API APOD de la NASA avec des informations sur les images
    (si elles sont disponibles) : résolution, mode, format, marque et modèle
    de l'appareil photo et logiciel de retouche utilisé.

    Paramètre :
    -
    - file_to_process : le fichier JSON à traiter.
    '''

    # Importe le fichier
    with open(file_to_process, 'r', encoding='utf-8') as f:
        data = json.load(f)

    if data['media_type'] == 'image':

        # Ouvre l'image à partir de 'media_url'
        response = requests.get(data['media_url'])
        
        if response.status_code != 404:
            img = Image.open(BytesIO(response.content))

            # Ajoute les informations de base sur l'image
            data['img_width_px'] = img.size[0]
            data['img_height_px'] = img.size[1]
            data['img_mode'] = img.mode
            data['img_format'] = img.format

            # Vérifie si l'image a des données EXIF
            if hasattr(img, '_getexif') and img._getexif():
                # Extrait les données EXIF
                exif_data = img._getexif()

                # Définit les tags à extraire
                tags_to_extract = ['Make', 'Model', 'Software']

                # Rechercher les tags dans les données EXIF
                extracted_tags = {
                    ExifTags.TAGS.get(tag_id, tag_id): value 
                    for tag_id, value in exif_data.items()
                    if ExifTags.TAGS.get(tag_id) in tags_to_extract
                }

                # Ajoute les informations dictionnaire
                for tag_name, value in extracted_tags.items():
                    data[tag_name] = value

    # Renomme les clés
    if 'Make' in data:
        data['camera_make'] = data.pop('Make', None)
    if 'Model' in data:
        data['camera_model'] = data.pop('Model', None)
    if 'Software' in data:
        data['software'] = data.pop('Software', None)

    # Définit les clés pour lesquelles nettoyer le texte
    keys_to_clean = ['camera_make', 'camera_model']

    # Boucle sur les clés à nettoyer et applique les règles définies
    for key in keys_to_clean:
        if key in data:
            # Supprime le caractère '\x00'
            data[key] = data[key].replace('\x00', '')
            # Supprime les espaces superflus
            data[key] = re.sub(r'\s+', ' ', data[key]) 
            # Supprimer l'espace à la fin de la chaîne de caractères
            data[key] = data[key].rstrip()
    
    # Exporte le fichier dans le dossier de destination
    p = Path.cwd()
    q = p / JSON_FOLDER / LEVEL1_FOLDER / file_to_process.name

    with open(q, 'w', encoding='utf-8') as of: 
        json.dump(data, of, indent=4)

### Ajout des information relative au texte

Pour chaque fichier enregistré au *niveau level 1*, on utilise **spaCy** pour extraire les mots clés et les entités de localisation contenus dans la variable `explanation`. Un nouveau fichier est ensuite enregistré au *niveau level 2*.

In [7]:
def generate_keywords_and_locations(file_to_process, x=20):
    '''
    Fonction permettant d'enrichir les données des fichiers JSON issus des
    requêtes sur l'API APOD de la NASA en extrayant les mots clés et les
    entités nommées (lieux) à partir de la variable 'explanation'.

    Paramètres :
    -
    - file_to_process : le fichier JSON à traiter ;
    - x : le nombre de mots-clés à conserver dans le top (20 par défaut).
    '''

    # Importe le fichier
    with open(file_to_process, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    # Définit le texte à analyser
    text = data['explanation']

    # Charge le modèle linguistique en anglais
    nlp = spacy.load('en_core_web_sm')

    # Traite le texte avec spaCy
    doc = nlp(text)

    # I. Récupère les entités nommées de localisation

    # Initialise un set
    named_entities = set()

    # Ajoute les entités géopolitiques et les lieux au set
    [named_entities.add(ent.text)
        for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]

    # Ajoute le set au dictionnaire 'data'
    if len(named_entities) > 0:
        data['named_entities'] = (', ').join(sorted(named_entities))
    
    # II. Récupère les mots clés

    # Initialise un dictionnaire
    keywords = {}

    # Boucle sur les 'tokens' du texte
    for token in doc:

        # Filtre les 'tokens' en fonction de leur nature
        if (
            token.is_alpha
            and not token.is_stop
            and token.pos_ != 'ADV'
            and token.ent_type_ not in ['GPE', 'LOC']
        ):

            # Utilise le lemme du mot
            keyword = token.lemma_

            # Ajoute le mot et son score au dictionnaire 'keywords' s'il n'est
            # pas déjà présent ou s'il a un score plus élevé
            if keyword not in keywords or token.rank > keywords[keyword]:
                keywords[keyword] = token.rank

    # Trie les mots-clés selon leur score
    sorted_keywords = sorted(
        keywords.items(), key=lambda x: x[1], reverse=True)

    # Sélectionne les 'x' premiers résultats
    top_keywords = [keyword[0] for keyword in sorted_keywords[:x]]

    # Ajoute la liste au dictionnaire 'data'
    if len(top_keywords) > 0:
        data['keywords'] = (', ').join(top_keywords)

    # III. Enregistrement du JSON

    # Exporte le fichier dans le dossier de destination
    p = Path.cwd()
    q = p / JSON_FOLDER / LEVEL2_FOLDER / file_to_process.name

    with open(q, 'w', encoding='utf-8') as of: 
        json.dump(data, of, indent=4)

### Enregistrement du *dataset* au format CSV

Tous les fichiers du *niveau level 2* sont compilés au sein d'un même *DataFrame* puis celui-ci est exporté en fichier CSV.

In [8]:
def csv_dataset_from_json_files(csv_filename='nasa-apod-dataset.csv'):
    
    '''
    Fonction permettant de créer et d'exporter un dataset au format CSV à 
    partir des fichiers JSON issus des requêtes sur l'API APOD de la NASA qui 
    ont ensuite été nettoyés et enrichis.

    Paramètre :
    -
    - csv_filename : le nom du fichier CSV à créer (par défaut 
    'nasa-apod-dataset.csv').
    '''

    # Initialise un DataFrame
    df = pd.DataFrame(
        columns=[
            'date',
            'title',
            'copyright',
            'explanation',
            'keywords',
            'named_entities',
            'media_type',
            'media_url',
            'img_format',
            'img_mode',
            'img_width_px',
            'img_height_px',
            'camera_make',
            'camera_model',
            'software'
        ]
    )

    # Crée la liste des fichiers JSON a lire
    p = Path.cwd()
    q = p / JSON_FOLDER / LEVEL2_FOLDER
    files = Path(q).glob('*.json')

    # Charge un fichier 'json' puis l'ajoute au DataFrame
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)

            df = pd.concat(
                [df, pd.DataFrame.from_dict(data, orient='index').T],
                ignore_index=True
        )
            
    # Exporte le DataFrame dans un fichier 'csv'
    q = p / CSV_FOLDER / csv_filename
    df.to_csv(q, sep=';',  index=False, encoding='utf-8')

## Exécution du code

In [37]:
def main():

    # Récupère les données
    get_raw_json_from_apod_api(start_date='2024-01-01', end_date='2024-01-02')

    # Liste les fichiers JSON à traiter
    p = Path.cwd()
    q = p / JSON_FOLDER / RAW_FOLDER
    files = q.glob('*.json')

    # Applique les fonctions de nettoyage et enrichissement
    for file in files:
        clean_json_file(file.parent.parent / RAW_FOLDER / file.name)
        add_image_features_to_json(
            file.parent.parent / LEVEL0_FOLDER / file.name)
        generate_keywords_and_locations(
            file.parent.parent / LEVEL1_FOLDER / file.name)

In [38]:
# main()